In [1]:
# Import dependencies 
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
from config import gkey
 
from bs4 import BeautifulSoup as bs
import requests as req

from sqlalchemy import create_engine

In [2]:
# Read breweries csv files
breweries = "data/breweries.csv"
breweries_df = pd.read_csv(breweries)
# Rename columns for better readability
brewery_id_df = breweries_df.rename(columns={'Unnamed: 0': 'brewery_id', 'name': 'brewery_name'})
# Preview breweries df
brewery_id_df.head()

,brewery_id,brewery_name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [3]:
# Read beers csv files
beers = "data/beers.csv"
beers_df = pd.read_csv(beers)
# Rename columns for better readability
beers_id_df = beers_df.rename(columns={'Unnamed: 0': 'index', 'beer_id': 'id', 'name': 'beer_name'})
# Preview beers df
beers_id_df 

,index,abv,ibu,id,beer_name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0
...,...,...,...,...,...,...,...,...
2405,2405,0.067,45.0,928,Belgorado,Belgian IPA,424,12.0
2406,2406,0.052,NaN,807,Rail Yard Ale,American Amber / Red Ale,424,12.0
2407,2407,0.055,NaN,620,B3K Black Lager,Schwarzbier,424,12.0
2408,2408,0.055,40.0,145,Silverback Pale Ale,American Pale Ale (APA),424,12.0


In [4]:
# Merge beers and breweries df
merged_beers_df = pd.merge(brewery_id_df, beers_id_df , on="brewery_id")
merged_beers_df

,brewery_id,brewery_name,city,state,index,abv,ibu,id,beer_name,style,ounces
0,0,NorthGate Brewing,Minneapolis,MN,1493,0.045,50.0,2692,Get Together,American IPA,16.0
1,0,NorthGate Brewing,Minneapolis,MN,1494,0.049,26.0,2691,Maggie's Leap,Milk / Sweet Stout,16.0
2,0,NorthGate Brewing,Minneapolis,MN,1495,0.048,19.0,2690,Wall's End,English Brown Ale,16.0
3,0,NorthGate Brewing,Minneapolis,MN,1496,0.060,38.0,2689,Pumpion,Pumpkin Ale,16.0
4,0,NorthGate Brewing,Minneapolis,MN,1497,0.060,25.0,2688,Stronghold,American Porter,16.0
...,...,...,...,...,...,...,...,...,...,...,...
2405,556,Butternuts Beer and Ale,Garrattsville,NY,488,0.049,NaN,52,Heinnieweisse Weissebier,Hefeweizen,12.0
2406,556,Butternuts Beer and Ale,Garrattsville,NY,489,0.068,NaN,51,Snapperhead IPA,American IPA,12.0
2407,556,Butternuts Beer and Ale,Garrattsville,NY,490,0.049,NaN,50,Moo Thunder Stout,Milk / Sweet Stout,12.0
2408,556,Butternuts Beer and Ale,Garrattsville,NY,491,0.043,NaN,49,Porkslap Pale Ale,American Pale Ale (APA),12.0


In [5]:
# Extract region column and get all unique values
unique_brewery = merged_beers_df['brewery_name'].unique()
#print(unique_brewery)
merged_beers_df['brewery_name'].nunique()

551

In [6]:
# Iterate through each unique region and do API pull to get their longitude/latitudes and rating
lat = []
lng = []
rating = []
for brewery in unique_brewery:
    
    params = {"input": brewery, 
              "key": gkey}

    target_url = ('https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={0}&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key={1}').format(brewery, gkey) 
    
    response = requests.get(target_url).json()
    try:
        latitude = response['candidates'][0]['geometry']['location']['lat']
        longitude = response['candidates'][0]['geometry']['location']['lng']
        if response['candidates'][0]['rating'] > 0:
            rate = response['candidates'][0]['rating']
        else:
            latitude = float('NaN')
            longitude = float('NaN')
            rate = float('NaN')
        
        print(brewery + ' Brewery Found')
    except:
        latitude = float('NaN')
        longitude = float('NaN')
        rate = float('NaN')
        print(brewery + ': Brewery Not found')
    lat.append(latitude)
    lng.append(longitude)
    rating.append(rate)

NorthGate Brewing  Brewery Found
Against the Grain Brewery Brewery Found
Jack's Abby Craft Lagers Brewery Found
Mike Hess Brewing Company Brewery Found
Fort Point Beer Company Brewery Found
COAST Brewing Company Brewery Found
Great Divide Brewing Company Brewery Found
Tapistry Brewing Brewery Found
Big Lake Brewing Brewery Found
The Mitten Brewing Company Brewery Found
Brewery Vivant Brewery Found
Petoskey Brewing Brewery Found
Blackrocks Brewery Brewery Found
Perrin Brewing Company Brewery Found
Witch's Hat Brewing Company Brewery Found
Founders Brewing Company Brewery Found
Flat 12 Bierwerks Brewery Found
Tin Man Brewing Company Brewery Found
Black Acre Brewing Co. Brewery Found
Brew Link Brewing Brewery Found
Bare Hands Brewery Brewery Found
Three Pints Brewing Brewery Found
Four Fathers Brewing  Brewery Found
Indiana City Brewing Brewery Found
Burn 'Em Brewing Brewery Found
Sun King Brewing Company Brewery Found
Evil Czech Brewery Brewery Found
450 North Brewing Company Brewery Fou

Laughing Dog Brewing Company Brewery Found
Bozeman Brewing Company Brewery Found
Big Choice Brewing Brewery Found
Big Storm Brewing Company Brewery Found
Carton Brewing Company Brewery Found
Midnight Sun Brewing Company Brewery Found
Fat Head's Brewery Brewery Found
Refuge Brewery Brewery Found
Chatham Brewing Brewery Found
DC Brau Brewing Company Brewery Found
Geneva Lake Brewing Company Brewery Found
Rochester Mills Brewing Company Brewery Found
Cape Ann Brewing Company Brewery Found
Borderlands Brewing Company Brewery Found
College Street Brewhouse and Pub Brewery Found
Joseph James Brewing Company: Brewery Not found
Harpoon Brewery Brewery Found
Back East Brewing Company Brewery Found
Champion Brewing Company Brewery Found
Devil's Backbone Brewing Company Brewery Found
Newburgh Brewing Company Brewery Found
Wiseacre Brewing Company Brewery Found
Golden Road Brewing Brewery Found
New Republic Brewing Company Brewery Found
Infamous Brewing Company Brewery Found
Two Henrys Brewing Com

Kona Brewing Company Brewery Found
MillKing It Productions: Brewery Not found
Pateros Creek Brewing Company Brewery Found
O'Fallon Brewery Brewery Found
Marble Brewery Brewery Found
Big Wood Brewery Brewery Found
Howard Brewing Company Brewery Found
Downeast Cider House Brewery Found
Swamp Head Brewery Brewery Found
Mavericks Beer Company Brewery Found
TailGate Beer Brewery Found
Northwest Brewing Company Brewery Found
Dad & Dude's Breweria: Brewery Not found
Centennial Beer Company Brewery Found
Denali Brewing Company Brewery Found
Deschutes Brewery Brewery Found
Sunken City Brewing Company: Brewery Not found
The Black Tooth Brewing Company Brewery Found
Kenai River Brewing Company Brewery Found
River North Brewery Brewery Found
Fremont Brewing Company Brewery Found
Armstrong Brewing Company Brewery Found
AC Golden Brewing Company: Brewery Not found
Big Bend Brewing Company Brewery Found
Good Life Brewing Company Brewery Found
Engine 15 Brewing Brewery Found
Green Room Brewing Brewery

In [7]:
# Make a dictionary with brewery name as key and list of lat,lng as value
brewery_dict = {}
for i in range(len(unique_brewery)):
    brewery_dict[unique_brewery[i]] = [[lat[i], lng[i]], rating[i]]

pprint(brewery_dict)

{'10 Barrel Brewing Company': [[39.7594088, -104.9855212], 4.5],
 '18th Street Brewery': [[41.6151687, -87.51724980000002], 4.5],
 '2 Towns Ciderhouse': [[44.563505, -123.2261848], 4.8],
 '21st Amendment Brewery': [[37.7825242, -122.3924674], 4.3],
 '3 Daughters Brewing': [[27.7691552, -82.662814], 4.6],
 '4 Hands Brewing Company': [[38.6149554, -90.19752570000001], 4.6],
 '450 North Brewing Company': [[39.2678493, -85.8320882], 4.4],
 '7 Seas Brewing Company': [[47.3272987, -122.5798707], 4.6],
 '7venth Sun': [[28.0100473, -82.4505911], 4.8],
 'AC Golden Brewing Company': [[nan, nan], nan],
 'Abita Brewing Company': [[30.482108, -90.05501799999999], 4.6],
 'Against The Grain Brewery': [[38.2554896, -85.7440216], 4.4],
 'Against the Grain Brewery': [[38.2554896, -85.7440216], 4.4],
 'Airways Brewing Company': [[nan, nan], nan],
 'Alameda Brewing': [[37.7683926, -122.2387777], 4.7],
 'Alaskan Brewing Company': [[58.35757339999999, -134.4863484], 4.8],
 'Ale Asylum': [[43.120465, -89.354

 'Marshall Wharf Brewing Company': [[44.4289785, -69.0053707], 4.6],
 'Martin House Brewing Company': [[32.7636231, -97.310063], 4.7],
 'Matt Brewing Company': [[43.10444589999999, -75.2455361], 4.6],
 'Maui Brewing Company': [[20.7493371, -156.4380209], 4.5],
 'Mavericks Beer Company': [[43.6450071, -79.4028996], 3.8],
 "Meckley's Cidery": [[42.0589973, -84.4058646], 4.5],
 'Mehana Brewing Co.': [[nan, nan], nan],
 'Miami Brewing Company': [[25.4855882, -80.54043519999999], 4.2],
 "Mickey Finn's Brewery": [[42.2862266, -87.95469279999999], 4.1],
 'Midnight Sun Brewing Company': [[61.1472309, -149.8445081], 4.6],
 'Mike Hess Brewing Company': [[32.7477013, -117.1285406], 4.6],
 'Mikerphone Brewing': [[42.0297529, -87.96754530000001], 4.7],
 'Mikkeller': [[40.7570994, -73.8443645], 4.5],
 'MillKing It Productions': [[nan, nan], nan],
 'Milwaukee Brewing Company': [[43.0453548, -87.9222834], 4.3],
 'Minhas Craft Brewery': [[42.5998511, -89.64113309999999], 4.5],
 'Mission Brewery': [[32.

In [8]:
# Iterate through each row of original df to find their respective coords
lat2 = []
lng2 = []
rating2 = []
for index, row in merged_beers_df.iterrows():
    if row['brewery_name'] in brewery_dict:
        lat2.append(brewery_dict[row['brewery_name']][0][0])
        lng2.append(brewery_dict[row['brewery_name']][0][1])
        rating2.append(brewery_dict[row['brewery_name']][1])

In [9]:
# Append coords and ratings to original df
merged_beers_df['latitude'] = lat2
merged_beers_df['longitude'] = lng2
merged_beers_df['rating'] = rating2
merged_beers_df

,brewery_id,brewery_name,city,state,index,abv,ibu,id,beer_name,style,ounces,latitude,longitude,rating
0,0,NorthGate Brewing,Minneapolis,MN,1493,0.045,50.0,2692,Get Together,American IPA,16.0,44.998332,-93.221174,4.2
1,0,NorthGate Brewing,Minneapolis,MN,1494,0.049,26.0,2691,Maggie's Leap,Milk / Sweet Stout,16.0,44.998332,-93.221174,4.2
2,0,NorthGate Brewing,Minneapolis,MN,1495,0.048,19.0,2690,Wall's End,English Brown Ale,16.0,44.998332,-93.221174,4.2
3,0,NorthGate Brewing,Minneapolis,MN,1496,0.060,38.0,2689,Pumpion,Pumpkin Ale,16.0,44.998332,-93.221174,4.2
4,0,NorthGate Brewing,Minneapolis,MN,1497,0.060,25.0,2688,Stronghold,American Porter,16.0,44.998332,-93.221174,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405,556,Butternuts Beer and Ale,Garrattsville,NY,488,0.049,NaN,52,Heinnieweisse Weissebier,Hefeweizen,12.0,42.623539,-75.190988,4.6
2406,556,Butternuts Beer and Ale,Garrattsville,NY,489,0.068,NaN,51,Snapperhead IPA,American IPA,12.0,42.623539,-75.190988,4.6
2407,556,Butternuts Beer and Ale,Garrattsville,NY,490,0.049,NaN,50,Moo Thunder Stout,Milk / Sweet Stout,12.0,42.623539,-75.190988,4.6
2408,556,Butternuts Beer and Ale,Garrattsville,NY,491,0.043,NaN,49,Porkslap Pale Ale,American Pale Ale (APA),12.0,42.623539,-75.190988,4.6


In [10]:
# Convert 'abv' to percentages
merged_beers_df['abv_percent'] = merged_beers_df['abv'] * 100
merged_beers_df

# Drop columns that are not needed
dropped_beers_df = merged_beers_df.drop(columns=['abv', 'brewery_id', 'ounces', 'ibu', 'id', 'index'])
# Preview Data
dropped_beers_df


,brewery_name,city,state,beer_name,style,latitude,longitude,rating,abv_percent
0,NorthGate Brewing,Minneapolis,MN,Get Together,American IPA,44.998332,-93.221174,4.2,4.5
1,NorthGate Brewing,Minneapolis,MN,Maggie's Leap,Milk / Sweet Stout,44.998332,-93.221174,4.2,4.9
2,NorthGate Brewing,Minneapolis,MN,Wall's End,English Brown Ale,44.998332,-93.221174,4.2,4.8
3,NorthGate Brewing,Minneapolis,MN,Pumpion,Pumpkin Ale,44.998332,-93.221174,4.2,6.0
4,NorthGate Brewing,Minneapolis,MN,Stronghold,American Porter,44.998332,-93.221174,4.2,6.0
...,...,...,...,...,...,...,...,...,...
2405,Butternuts Beer and Ale,Garrattsville,NY,Heinnieweisse Weissebier,Hefeweizen,42.623539,-75.190988,4.6,4.9
2406,Butternuts Beer and Ale,Garrattsville,NY,Snapperhead IPA,American IPA,42.623539,-75.190988,4.6,6.8
2407,Butternuts Beer and Ale,Garrattsville,NY,Moo Thunder Stout,Milk / Sweet Stout,42.623539,-75.190988,4.6,4.9
2408,Butternuts Beer and Ale,Garrattsville,NY,Porkslap Pale Ale,American Pale Ale (APA),42.623539,-75.190988,4.6,4.3


In [11]:
# Drop NaN values
dropped_beers_df = dropped_beers_df.dropna(how='any')
dropped_beers_df

,brewery_name,city,state,beer_name,style,latitude,longitude,rating,abv_percent
0,NorthGate Brewing,Minneapolis,MN,Get Together,American IPA,44.998332,-93.221174,4.2,4.5
1,NorthGate Brewing,Minneapolis,MN,Maggie's Leap,Milk / Sweet Stout,44.998332,-93.221174,4.2,4.9
2,NorthGate Brewing,Minneapolis,MN,Wall's End,English Brown Ale,44.998332,-93.221174,4.2,4.8
3,NorthGate Brewing,Minneapolis,MN,Pumpion,Pumpkin Ale,44.998332,-93.221174,4.2,6.0
4,NorthGate Brewing,Minneapolis,MN,Stronghold,American Porter,44.998332,-93.221174,4.2,6.0
...,...,...,...,...,...,...,...,...,...
2405,Butternuts Beer and Ale,Garrattsville,NY,Heinnieweisse Weissebier,Hefeweizen,42.623539,-75.190988,4.6,4.9
2406,Butternuts Beer and Ale,Garrattsville,NY,Snapperhead IPA,American IPA,42.623539,-75.190988,4.6,6.8
2407,Butternuts Beer and Ale,Garrattsville,NY,Moo Thunder Stout,Milk / Sweet Stout,42.623539,-75.190988,4.6,4.9
2408,Butternuts Beer and Ale,Garrattsville,NY,Porkslap Pale Ale,American Pale Ale (APA),42.623539,-75.190988,4.6,4.3


In [13]:
# Replace 'Other' beer style with 'Fruit / Vegetable Beer'
replace_beer_df = dropped_beers_df['style'].replace('Other', 'Fruit / Vegetable Beer')
# Reset Index
indexed_beers_df = dropped_beers_df.reset_index()
# # Set Index
#final_beers_df = indexed_beers_df.set_index('index')
# Preview Data
indexed_beers_df

,index,brewery_name,city,state,beer_name,style,latitude,longitude,rating,abv_percent
0,0,NorthGate Brewing,Minneapolis,MN,Get Together,American IPA,44.998332,-93.221174,4.2,4.5
1,1,NorthGate Brewing,Minneapolis,MN,Maggie's Leap,Milk / Sweet Stout,44.998332,-93.221174,4.2,4.9
2,2,NorthGate Brewing,Minneapolis,MN,Wall's End,English Brown Ale,44.998332,-93.221174,4.2,4.8
3,3,NorthGate Brewing,Minneapolis,MN,Pumpion,Pumpkin Ale,44.998332,-93.221174,4.2,6.0
4,4,NorthGate Brewing,Minneapolis,MN,Stronghold,American Porter,44.998332,-93.221174,4.2,6.0
...,...,...,...,...,...,...,...,...,...,...
2250,2405,Butternuts Beer and Ale,Garrattsville,NY,Heinnieweisse Weissebier,Hefeweizen,42.623539,-75.190988,4.6,4.9
2251,2406,Butternuts Beer and Ale,Garrattsville,NY,Snapperhead IPA,American IPA,42.623539,-75.190988,4.6,6.8
2252,2407,Butternuts Beer and Ale,Garrattsville,NY,Moo Thunder Stout,Milk / Sweet Stout,42.623539,-75.190988,4.6,4.9
2253,2408,Butternuts Beer and Ale,Garrattsville,NY,Porkslap Pale Ale,American Pale Ale (APA),42.623539,-75.190988,4.6,4.3


In [14]:
# Save final DataFrame to CSV
indexed_beers_df.to_csv('final_beer.csv')